# Task 2
Petra Reuwsaat Paul C4CRVZYXW

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load data from Task 1
data = np.load('result.npz')
result = data['result']
neurons_preferred = data['neurons']
stimulus_orientations = data['stimulus_orientations']

# Check dimensions
n_neurons, n_stimuli, n_trials = result.shape
print(f"Shape: {n_neurons} neurons, {n_stimuli} orientations, {n_trials} trials")
print(f"Total trials to decode: {n_stimuli * n_trials}")

Shape: 50 neurons, 8 orientations, 30 trials
Total trials to decode: 240


## Part 1: Decode the Orientation

### 1.1 Organize data into DataFrame

In [3]:
# Reshape 3D array into long format DataFrame
# Following assignment instructions for columns
data_list = []

for neuron_id in range(n_neurons):
    for stim_idx in range(n_stimuli):
        for trial_idx in range(n_trials):
            data_list.append({
                'neuron_id': neuron_id,
                'stimulus_orientation': stimulus_orientations[stim_idx],
                'trial_number': trial_idx,
                'firing_rate': result[neuron_id, stim_idx, trial_idx],
                'preferred_angle': neurons_preferred[neuron_id]
            })

df = pd.DataFrame(data_list)
print(f"DataFrame shape: {df.shape}")
df.head()

DataFrame shape: (12000, 5)


,neuron_id,stimulus_orientation,trial_number,firing_rate,preferred_angle
0,0,0.0,0,75.0,152.819982
1,0,0.0,1,97.0,152.819982
2,0,0.0,2,98.0,152.819982
3,0,0.0,3,85.0,152.819982
4,0,0.0,4,95.0,152.819982


In [4]:
# Quick statistics check
df.describe()

,neuron_id,stimulus_orientation,trial_number,firing_rate,preferred_angle
count,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000
mean,24.500000,78.750000,14.500000,63.767250,88.329891
std,14.431471,51.556125,8.655802,31.872594,53.261128
min,0.000000,0.000000,0.000000,0.000000,0.383165
25%,12.000000,39.375000,7.000000,37.750000,37.063308
50%,24.500000,78.750000,14.500000,70.000000,87.991008
75%,37.000000,118.125000,22.000000,91.000000,128.352451
max,49.000000,157.500000,29.000000,140.000000,175.851976


### 1.2 Create decoding function

Using population vector method from assignment hint

In [5]:
def decode_orientation(firing_rates, preferred_angles):
    # Step 1: Double angles and convert to radians
    # Orientation wraps every 180 degrees so we double it
    angles_rad = np.radians(preferred_angles * 2)

    # Step 2: Calculate weighted sum for x and y components
    # Each neuron votes with its firing rate
    x_component = np.sum(firing_rates * np.cos(angles_rad))
    y_component = np.sum(firing_rates * np.sin(angles_rad))

    # Step 3: Get angle using arctan2
    decoded_rad = np.arctan2(y_component, x_component)

    # Step 4: Convert back to degrees and divide by 2
    decoded_deg = np.degrees(decoded_rad) / 2

    # Make sure angle is between 0 and 180
    if decoded_deg < 0:
        decoded_deg += 180

    return decoded_deg

### 1.3 Decode all trials

In [6]:
# Decode orientation for each trial
decoded_list = []

for stim_idx in range(n_stimuli):
    for trial_idx in range(n_trials):
        # Get firing rates for all neurons in this trial
        firing_rates = result[:, stim_idx, trial_idx]

        # Decode using population vector
        decoded = decode_orientation(firing_rates, neurons_preferred)
        decoded_list.append(decoded)

# Create DataFrame with results
df_decoded = pd.DataFrame({
    'stimulus_orientation': np.repeat(stimulus_orientations, n_trials),
    'trial_number': np.tile(range(n_trials), n_stimuli),
    'decoded_orientation': decoded_list
})

print(f"Decoded {len(decoded_list)} trials")
df_decoded.head(10)

Decoded 240 trials


,stimulus_orientation,trial_number,decoded_orientation
0,0.0,0,1.179583
1,0.0,1,1.457612
2,0.0,2,1.509759
3,0.0,3,0.977455
4,0.0,4,0.016577
5,0.0,5,1.682338
6,0.0,6,1.959100
7,0.0,7,0.870668
8,0.0,8,1.751267
9,0.0,9,1.394243


## Part 2: Estimate Your Calculation

In [7]:
# Calculate absolute error
diff = np.abs(df_decoded['stimulus_orientation'] - df_decoded['decoded_orientation'])
df_decoded['absolute_error'] = np.minimum(diff, 180 - diff)

# Compute average error
mean_error = df_decoded['absolute_error'].mean()
std_error = df_decoded['absolute_error'].std()

print(f"Mean Absolute Error: {mean_error:.2f} degrees")
print(f"Standard Deviation: {std_error:.2f} degrees")

Mean Absolute Error: 2.28 degrees
Standard Deviation: 1.74 degrees


In [8]:
# Show examples of decoded vs true orientations
print("First 10 trials:")
print(df_decoded[['stimulus_orientation', 'decoded_orientation', 'absolute_error']].head(10))

First 10 trials:
   stimulus_orientation  decoded_orientation  absolute_error
0                   0.0             1.179583        1.179583
1                   0.0             1.457612        1.457612
2                   0.0             1.509759        1.509759
3                   0.0             0.977455        0.977455
4                   0.0             0.016577        0.016577
5                   0.0             1.682338        1.682338
6                   0.0             1.959100        1.959100
7                   0.0             0.870668        0.870668
8                   0.0             1.751267        1.751267
9                   0.0             1.394243        1.394243


In [9]:
# Error by stimulus orientation
print("\nAverage error for each orientation:")
error_by_orientation = df_decoded.groupby('stimulus_orientation')['absolute_error'].mean()
print(error_by_orientation)


Average error for each orientation:
stimulus_orientation
0.0      1.540434
22.5     3.069573
45.0     4.117375
67.5     1.506196
90.0     0.782353
112.5    1.472114
135.0    1.046782
157.5    4.743797
Name: absolute_error, dtype: float64


## Part 3: Critical Thinking

### 3.1 Top 10 neurons vs all 50 neurons

In [10]:
# Find the 10 most responsive neurons
# Using mean firing rate across all stimuli and trials
mean_firing_rates = result.mean(axis=(1, 2))
top_10_neurons = np.argsort(mean_firing_rates)[-10:]

print(f"Top 10 most responsive neurons: {top_10_neurons}")
print(f"Their mean firing rates: {mean_firing_rates[top_10_neurons]}")

Top 10 most responsive neurons: [26  5 32  3 46 13 20 29 24 17]
Their mean firing rates: [64.24166667 64.24166667 64.36666667 64.4375     64.48333333 64.4875
 64.59583333 64.75416667 64.85833333 64.88333333]


In [11]:
# Decode using only top 10 neurons
decoded_top10 = []

for stim_idx in range(n_stimuli):
    for trial_idx in range(n_trials):
        # Only use top 10 neurons
        firing_rates_top10 = result[top_10_neurons, stim_idx, trial_idx]
        preferred_top10 = neurons_preferred[top_10_neurons]

        decoded = decode_orientation(firing_rates_top10, preferred_top10)
        decoded_top10.append(decoded)

print(f"Decoded {len(decoded_top10)} trials using only top 10 neurons")

Decoded 240 trials using only top 10 neurons


In [12]:
# Calculate error for top 10 neurons
df_top10 = pd.DataFrame({
    'stimulus_orientation': np.repeat(stimulus_orientations, n_trials),
    'decoded_orientation': decoded_top10
})

diff_top10 = np.abs(df_top10['stimulus_orientation'] - df_top10['decoded_orientation'])
df_top10['absolute_error'] = np.minimum(diff_top10, 180 - diff_top10)
mean_error_top10 = df_top10['absolute_error'].mean()

print(f"\nComparison:")
print(f"Error with ALL 50 neurons: {mean_error:.2f} degrees")
print(f"Error with TOP 10 neurons: {mean_error_top10:.2f} degrees")
print(f"Difference: {mean_error_top10 - mean_error:.2f} degrees")


Comparison:
Error with ALL 50 neurons: 2.28 degrees
Error with TOP 10 neurons: 7.19 degrees
Difference: 4.90 degrees


### 3.2 Effect of doubling noise

In [13]:
# Function from Task 1 to generate responses
def tuning_curve(stimulus, preferred, r_max=100):
    diff = abs(stimulus - preferred)
    diff = np.minimum(diff, 180 - diff)
    diff_rad = np.radians(diff)
    response = r_max * np.cos(diff_rad)
    return max(0, response)

# Generate new data with 2x noise (multiply Poisson parameter by 2)
result_high_noise = np.zeros((n_neurons, n_stimuli, n_trials))

for i in range(n_neurons):
    for j in range(n_stimuli):
        expected_rate = tuning_curve(stimulus_orientations[j], neurons_preferred[i])
        for k in range(n_trials):
            # Double the Poisson parameter = 2x noise
            result_high_noise[i, j, k] = np.random.poisson(expected_rate * 2)

print("Generated new data with doubled noise")

Generated new data with doubled noise


In [14]:
# Decode with high noise data
decoded_noise = []

for stim_idx in range(n_stimuli):
    for trial_idx in range(n_trials):
        firing_rates_noise = result_high_noise[:, stim_idx, trial_idx]
        decoded = decode_orientation(firing_rates_noise, neurons_preferred)
        decoded_noise.append(decoded)

print(f"Decoded {len(decoded_noise)} trials with doubled noise")

Decoded 240 trials with doubled noise


In [15]:
# Calculate error with doubled noise
df_noise = pd.DataFrame({
    'stimulus_orientation': np.repeat(stimulus_orientations, n_trials),
    'decoded_orientation': decoded_noise
})

diff_noise = np.abs(df_noise['stimulus_orientation'] - df_noise['decoded_orientation'])
df_noise['absolute_error'] = np.minimum(diff_noise, 180 - diff_noise)
mean_error_noise = df_noise['absolute_error'].mean()

print(f"\nComparison:")
print(f"Normal noise: {mean_error:.2f} degrees")
print(f"Doubled noise (2x): {mean_error_noise:.2f} degrees")
print(f"Difference: {mean_error_noise - mean_error:.2f} degrees")


Comparison:
Normal noise: 2.28 degrees
Doubled noise (2x): 2.31 degrees
Difference: 0.03 degrees


## Conclusions

Question 3.1: Top 10 vs All 50 neurons

Using only the top 10 most responsive neurons increased the decoding error compared to using all 50 neurons. This shows that population coding benefits from redundancy, even neurons with lower firing rates contribute useful information. The population vector method works better with more neurons because averaging across a larger population reduces the impact of individual neuron variability.

Question 3.2: Effect of doubling noise

Doubling the noise level (2× Poisson parameter) increased the decoding error slightly. However, the increase was relatively small, demonstrating that population vector decoding is robust to noise when using many neurons. Averaging across 50 neurons helps cancel out random variations, making the decoding more reliable even with higher noise levels.